In [2]:
!unzip /content/drive/MyDrive/esml/anomaly.zip -d dataset1

Archive:  /content/drive/MyDrive/esml/anomaly.zip
   creating: dataset1/pre_video/
   creating: dataset1/pre_video/falling/
  inflating: dataset1/pre_video/falling/okutama_2.2.8_Lying_21.mp4  
  inflating: dataset1/pre_video/falling/okutama_2.1.2_Lying_48.mp4  
  inflating: dataset1/pre_video/falling/okutama_1.1.7_Lying_29.mp4  
  inflating: dataset1/pre_video/falling/okutama_2.2.10_Lying_33.mp4  
  inflating: dataset1/pre_video/falling/casia_horizontalview_p08_faint_a1.mp4  
  inflating: dataset1/pre_video/falling/casia_horizontalview_p09_faint_a1.mp4  
  inflating: dataset1/pre_video/falling/okutama_2.1.1_Lying_46.mp4  
  inflating: dataset1/pre_video/falling/okutama_2.2.2_Lying_1.mp4  
  inflating: dataset1/pre_video/falling/okutama_1.2.10_Lying_2.mp4  
  inflating: dataset1/pre_video/falling/okutama_1.2.6_Lying_6.mp4  
  inflating: dataset1/pre_video/falling/okutama_2.1.1_Lying_1.mp4  
  inflating: dataset1/pre_video/falling/okutama_1.1.4_Lying_27.mp4  
  inflating: dataset1/pre_vi

In [2]:
import os
import shutil
import random

In [3]:
src_dir = "/content/dataset1/anomaly"

dst_dir = "/content/dataset1"

os.makedirs(os.path.join(dst_dir, "train"), exist_ok=True)
os.makedirs(os.path.join(dst_dir, "test"), exist_ok=True)

In [4]:
!rm -r /content/dataset1/anomaly/abuse
!rm -r /content/dataset1/anomaly/breakin_entry
!rm -r /content/dataset1/anomaly/explosion

rm: cannot remove '/content/dataset1/anomaly/abuse': No such file or directory
rm: cannot remove '/content/dataset1/anomaly/breakin_entry': No such file or directory
rm: cannot remove '/content/dataset1/anomaly/explosion': No such file or directory


In [8]:
for directory in os.listdir(src_dir):

  dst1_path = os.path.join(dst_dir, "train", directory)
  dst2_path = os.path.join(dst_dir, "test", directory)

  if not os.path.isdir(dst1_path):
    os.makedirs(dst1_path)
  if not os.path.isdir(dst2_path):
    os.makedirs(dst2_path)

  list_files = os.listdir(os.path.join(src_dir, directory))
  random.shuffle(list_files)
  num_files = len(list_files)
  num_train = int(num_files * 0.8)
  for i, file_name in enumerate(list_files):
    if i < num_train:
      dst_path = dst1_path
    else:
      dst_path = dst2_path
    shutil.copy(os.path.join(src_dir, directory, file_name), os.path.join(dst_path, file_name))

In [9]:
rm -r /content/dataset1/anomaly

In [5]:
# train data

for directory in os.listdir(os.path.join(dst_dir, "train")):
  print(directory, len(os.listdir(os.path.join(dst_dir, "train", directory))))

panicking 45
human_entering_door 53
arson 40
murdering_weapon_violence 164
vandalizing 172
steeling_robbary 271
hitting_fighting 406
falling 98
kicking 64


In [6]:
# test data

for directory in os.listdir(os.path.join(dst_dir, "test")):
  print(directory, len(os.listdir(os.path.join(dst_dir, "test", directory))))

panicking 12
human_entering_door 14
arson 10
murdering_weapon_violence 41
vandalizing 43
steeling_robbary 68
hitting_fighting 102
falling 25
kicking 17


In [7]:
# package import
import os
import cv2
import tqdm
import time
import shutil
import json
import random
import pathlib
from pathlib import PosixPath
import itertools
import collections
import numpy as np
from IPython import display
from urllib import request

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from tensorflow.keras.applications import VGG16, InceptionResNetV2, MobileNetV2
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import to_categorical

In [8]:
# InceptionResNetV2
# Input shape to be used by the model
# pInputShape = (229, 229, 3)
# pNumFrames  = 30
# pModelName  = "InceptionResNetV2"
# pStrategyForFrame = "smart_select"
# pBatchSize  = 2
# pFeatureExtractorOutShape = 1536



# MobileNetV2
# # Input shape to be used by the model
# pInputShape = (224, 224, 3)
# pNumFrames  = 30
# pStrategyForFrame = "smart_select"
# pBatchSize  = 2
# pFeatureExtractorOutShape = 1280


main_path = "/content/dataset1"
train_data_path = "/content/dataset1/train"
test_data_path = "/content/dataset1/test"

In [9]:
def convert_to_one_hot(labels, num_classes=None):
    """
    Convert integer labels to one-hot encoded labels.

    Parameters:
    - labels: Array-like. Integer labels.
    - num_classes: int, optional. The total number of classes. If not provided, it will be inferred from the maximum label value.

    Returns:
    - One-hot encoded labels as a numpy array.
    """
    return to_categorical(labels, num_classes=num_classes)

In [10]:
import tensorflow as tf
import cv2
import random
import numpy as np


class FrameGenerator:
    def __init__(self, path, n_frames, output_size, frame_step, selection_strategy, training = False):
        """ Returns a set of frames with their associated label.

          Args:
            path: Video file paths.
            n_frames: Number of frames.
            selection_strategy: Provide which startegy to use for frame extraction
            we have support for 3 different stategies: 1. random, 2. dynamic and 3. smart_select
            training: Boolean to determine if training dataset is being created.
        """
        # print("path", path)
        self.path = path
        self.n_frames = n_frames
        self.output_size = output_size
        self.frame_step = frame_step
        self.selection_strategy = selection_strategy
        self.training = training
        self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
        self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

        print("self.class_names ", self.class_names)
        print("self.class_ids_for_name ", self.class_ids_for_name)


    # Function that receive the video frame and output the padded and resized frame.
    def format_frames(self, frame):
        """
        Pad and resize an image from a video.

        Args:
        frame: Image that needs to resized and padded.
        output_size: Pixel size of the output frame image.

        Return:
        Formatted frame with padding of specified output size.
        """
        try:
            frame = tf.image.convert_image_dtype(frame, tf.float32)
            frame = tf.image.resize_with_pad(frame, *self.output_size)
            return frame

        except tf.errors.InvalidArgumentError as e:
            # Handle the specific exception raised when the conversion fails
            print(f"Error converting image data type: {e}")
        except Exception as e:
            # Handle other exceptions that might occur
            print(f"An unexpected error occurred: {e}")


    # Function that convert the received input video and output constant number of frames.
    def frames_from_video_file(self, video_path):
        """
        Creates frames from each video file present for each category.

        Args:
        video_path: File path to the video.
        selection_strategy: Provide which startegy to use for frame extraction
        n_frames: Number of frames to be created per video file.
        output_size: Pixel size of the output frame image.

        Return:
        An NumPy array of frames in the shape of (n_frames, height, width, channels).
        """
        # Read each video frame by frame
        result = []
        src = cv2.VideoCapture(str(video_path))

        src.set(cv2.CAP_PROP_POS_FRAMES, 0)
        # ret is a boolean indicating whether read was successful, frame is the image itself
        for _ in range(self.n_frames):
            ret, frame = src.read()
            if ret:
                frame = self.format_frames(frame)
                result.append(frame)
            else:
                frame = np.zeros((256, 256, 3), dtype = np.uint8)
                frame = self.format_frames(frame)
                result.append(frame)
            src.release()

        return np.array(result)[..., [2, 1, 0]]

    def get_files_and_class_names(self):
        video_paths = list(self.path.glob('*/*.mp4'))
        classes = [p.parent.name for p in video_paths]
        return video_paths, classes

    def __call__(self):
        video_paths, classes = self.get_files_and_class_names()

        pairs = list(zip(video_paths, classes))

        if self.training:
            random.shuffle(pairs)

        for path, name in pairs:
            video_frames = self.frames_from_video_file(video_path = path)
            label = self.class_ids_for_name[name] # Encode labels
            yield video_frames, convert_to_one_hot(label, NUM_CLASSES)


In [22]:
######################################################################################################################################################
# check whether we GPU is available or not
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# define number of classes and files per class
NUM_CLASSES = len(os.listdir(train_data_path))
print("Number of classes: ", NUM_CLASSES)


# Get total number of video's across train and test
mainPath = pathlib.Path(main_path)
video_count_train = len(list(mainPath.glob('train/*/*.mp4')))
video_count_test = len(list(mainPath.glob('test/*/*.mp4')))
video_total = video_count_train + video_count_test
print(f"Total train videos: {video_count_train}")
print(f"Total test videos: {video_count_test}")
print(f"Total videos: {video_total}")
######################################################################################################################################################

Num GPUs Available:  1
Number of classes:  9
Total train videos: 1313
Total test videos: 332
Total videos: 1645


In [23]:
######################################################################################################################################################
# custom frame selection class, to select the best frames from the given video
# from frame_generator import FrameGenerator
pNumFrames = 60
pInputShape = (224, 224, 3)
pStrategyForFrame = "smart_select"
pBatchSize = 8

subset_paths = {}
subset_paths['train'] = PosixPath(os.path.join(main_path, 'train'))
subset_paths['test'] = PosixPath(os.path.join(main_path, 'test'))

fg = FrameGenerator(path = subset_paths['train'], n_frames = pNumFrames, output_size = pInputShape[:2],
                    frame_step = 5, selection_strategy = pStrategyForFrame, training = True)

frames, label = next(fg())

print(f"Shape: {frames.shape}")
print(f"Label: {label}")
######################################################################################################################################################

self.class_names  ['arson', 'falling', 'hitting_fighting', 'human_entering_door', 'kicking', 'murdering_weapon_violence', 'panicking', 'steeling_robbary', 'vandalizing']
self.class_ids_for_name  {'arson': 0, 'falling': 1, 'hitting_fighting': 2, 'human_entering_door': 3, 'kicking': 4, 'murdering_weapon_violence': 5, 'panicking': 6, 'steeling_robbary': 7, 'vandalizing': 8}
Shape: (60, 224, 224, 3)
Label: [0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [24]:
######################################################################################################################################################
# Create the training set
output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (NUM_CLASSES), dtype = tf.int16))
train_ds = tf.data.Dataset.from_generator(FrameGenerator(path = subset_paths['train'], n_frames = pNumFrames, output_size = pInputShape[:2], frame_step = 5, selection_strategy = pStrategyForFrame, training = True),
                                         output_signature = output_signature)


for frames, labels in train_ds.take(10):
    print(labels)

# Create the validation set
val_ds = tf.data.Dataset.from_generator(FrameGenerator(path = subset_paths['test'], n_frames = pNumFrames, output_size = pInputShape[:2], frame_step = 5, selection_strategy = pStrategyForFrame, training = False),
                                        output_signature = output_signature)

# Print the shapes of the data
train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

val_frames, val_labels = next(iter(val_ds))
print(f'Shape of validation set of frames: {val_frames.shape}')
print(f'Shape of validation labels: {val_labels.shape}')
######################################################################################################################################################


AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(10).prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.shuffle(10).prefetch(buffer_size = AUTOTUNE)

train_ds = train_ds.batch(pBatchSize)
val_ds = val_ds.batch(pBatchSize)

train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

val_frames, val_labels = next(iter(val_ds))
print(f'Shape of validation set of frames: {val_frames.shape}')
print(f'Shape of validation labels: {val_labels.shape}')

self.class_names  ['arson', 'falling', 'hitting_fighting', 'human_entering_door', 'kicking', 'murdering_weapon_violence', 'panicking', 'steeling_robbary', 'vandalizing']
self.class_ids_for_name  {'arson': 0, 'falling': 1, 'hitting_fighting': 2, 'human_entering_door': 3, 'kicking': 4, 'murdering_weapon_violence': 5, 'panicking': 6, 'steeling_robbary': 7, 'vandalizing': 8}
tf.Tensor([0 0 1 0 0 0 0 0 0], shape=(9,), dtype=int16)
tf.Tensor([0 0 0 0 0 0 0 1 0], shape=(9,), dtype=int16)
tf.Tensor([0 0 0 0 0 0 0 1 0], shape=(9,), dtype=int16)
tf.Tensor([0 0 0 0 0 1 0 0 0], shape=(9,), dtype=int16)
tf.Tensor([0 0 0 0 0 1 0 0 0], shape=(9,), dtype=int16)
tf.Tensor([0 0 0 0 0 1 0 0 0], shape=(9,), dtype=int16)
tf.Tensor([0 0 1 0 0 0 0 0 0], shape=(9,), dtype=int16)
tf.Tensor([0 0 0 0 1 0 0 0 0], shape=(9,), dtype=int16)
tf.Tensor([0 0 0 0 0 0 0 0 1], shape=(9,), dtype=int16)
tf.Tensor([0 0 0 0 0 0 1 0 0], shape=(9,), dtype=int16)
self.class_names  ['arson', 'falling', 'hitting_fighting', 'human_

In [15]:
def weighted_categorical_crossentropy(y_true, y_pred, class_weights):
    # Calculate categorical crossentropy
    cce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)

    # print(cce)
    # Apply class weights
    # weighted_cce = tf.reduce_mean(tf.multiply(cce, class_weights))

    return cce  #weighted_cce


def get_the_class_weights(class_names):
    weights = np.ones(NUM_CLASSES)
    # for idx, class_name in enumerate(class_names):
    #     weights[idx] = video_count_train / len(os.listdir(os.path.join(train_data_path, class_name)))
    return tf.constant(weights)


class_names = fg.class_names
class_weights = get_the_class_weights(class_names)
print("class_weights", class_weights)
######################################################################################################################################################

print("train_ds.class_names", )

class_weights tf.Tensor([1. 1. 1. 1. 1. 1. 1. 1. 1.], shape=(9,), dtype=float64)
train_ds.class_names


In [16]:
# # write a custom class balanced loss function
# import tensorflow as tf

# class CategoricalWeightedCrossEntropyLoss(tf.keras.losses.Loss):

#     def __init__(self, name='categorical_weighted_cross_entropy_loss'):
#         self.class_weights = class_weights
#         super(CategoricalWeightedCrossEntropyLoss, self).__init__()

#     def call(self, y_true, y_pred):
#         epsilon = 1e-15
#         y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)
#         temp_loss = tf.cast(y_true, dtype=tf.float32)  * tf.cast(tf.math.log(y_pred), dtype=tf.float32)
#         weighted_loss = tf.cast(self.class_weights, dtype=tf.float32) * temp_loss
#         loss = -tf.reduce_mean(weighted_loss)
#         return loss

# # Create an instance of your custom loss class
# custom_loss = CategoricalWeightedCrossEntropyLoss(class_weights)

# from functools import partial

# # Use functools.partial to pass y_weights to the loss function
# loss_function = partial(custom_loss)

In [25]:
######################################################################################################################################################

METRICS = [
    #   keras.metrics.BinaryCrossentropy(name='cross entropy'),  # same as model's loss
    #   keras.metrics.MeanSquaredError(name='Brier score'),
      # keras.metrics.TruePositives(name='tp'),
      # keras.metrics.FalsePositives(name='fp'),
      # keras.metrics.TrueNegatives(name='tn'),
      # keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.CategoricalAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [26]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger


input_shape = (pBatchSize, pNumFrames, 224, 224, 3)


model = tf.keras.Sequential([
    Conv3D(16, (3, 3, 3), input_shape = input_shape[1:], activation='relu'),
    MaxPooling3D((2, 2, 2)),
    Conv3D(32, (3, 3, 3), activation='relu'),
    MaxPooling3D((2, 2, 2)),
    Conv3D(64, (3, 3, 3), activation='relu'),
    MaxPooling3D((2, 2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')  # 9 classes for anomaly detection
])

In [31]:
idx = 0
a = [idx]*19
print(a)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [43]:
# calculate class weights
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os
class_weigths = {}
y = []
for idx, directory in enumerate(fg.class_names):
  num = len(os.listdir(os.path.join(dst_dir, "train", directory)))
  class_weigths[directory] = num
  y.extend([idx] * num)

classes = np.arange(0, 9)
print(class_weigths)
print(len(y))
print(y)

import sklearn
skclass_weights = compute_class_weight('balanced', classes = classes, y = y)
print(skclass_weights)

{'arson': 40, 'falling': 98, 'hitting_fighting': 406, 'human_entering_door': 53, 'kicking': 64, 'murdering_weapon_violence': 164, 'panicking': 45, 'steeling_robbary': 271, 'vandalizing': 172}
1313
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

In [44]:
# from tf.keras.losses import CategoricalFocalCrossentropy
# Using 'sum' reduction type.
categorical_focal_ce_loss = tf.keras.losses.CategoricalFocalCrossentropy(
    alpha=skclass_weights, gamma=2,
    from_logits=False, reduction=tf.keras.losses.Reduction.SUM)

In [45]:
# Compile the model
model.compile(optimizer='adam', loss = categorical_focal_ce_loss, metrics=METRICS)

print(model.summary())

# Define callbacks
checkpoint_callback = ModelCheckpoint("/content/drive/MyDrive/esml/9_class_classifier_with_sk_weighted_loss/model_epoch_{epoch:02d}.h5", save_freq='epoch')  # Save model at the end of each epoch
csv_logger = CSVLogger("/content/drive/MyDrive/esml/9_class_classifier_with_sk_weighted_loss/training_history.csv", append=True)  # Save training history to CSV file


EPOCHS = 10

# Train the model
model.fit(train_ds,
          epochs = EPOCHS,
          validation_data = val_ds,
          batch_size = pBatchSize,
          callbacks=[checkpoint_callback, csv_logger])
        #   callbacks = tf.keras.callbacks.EarlyStopping(patience = 2, monitor = 'val_loss'))
# # Print the model summary
# model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_3 (Conv3D)           (None, 58, 222, 222, 16   1312      
                             )                                   
                                                                 
 max_pooling3d_3 (MaxPoolin  (None, 29, 111, 111, 16   0         
 g3D)                        )                                   
                                                                 
 conv3d_4 (Conv3D)           (None, 27, 109, 109, 32   13856     
                             )                                   
                                                                 
 max_pooling3d_4 (MaxPoolin  (None, 13, 54, 54, 32)    0         
 g3D)                                                            
                                                                 
 conv3d_5 (Conv3D)           (None, 11, 52, 52, 64)   

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


165/165 [==============================] - 721s 4s/step - loss: 1.1693 - accuracy: 0.8213 - precision: 0.8467 - recall: 0.7891 - auc: 0.9786 - prc: 0.9000 - val_loss: 10.3026 - val_accuracy: 0.5723 - val_precision: 0.6034 - val_recall: 0.5271 - val_auc: 0.8961 - val_prc: 0.5860
Epoch 2/10
165/165 [==============================] - 714s 4s/step - loss: 0.7212 - accuracy: 0.9040 - precision: 0.9179 - recall: 0.8850 - auc: 0.9951 - prc: 0.9697 - val_loss: 10.2289 - val_accuracy: 0.5663 - val_precision: 0.5964 - val_recall: 0.5030 - val_auc: 0.9093 - val_prc: 0.6245
Epoch 3/10
165/165 [==============================] - 720s 4s/step - loss: 0.7454 - accuracy: 0.9056 - precision: 0.9186 - recall: 0.8858 - auc: 0.9951 - prc: 0.9676 - val_loss: 10.7810 - val_accuracy: 0.5813 - val_precision: 0.6159 - val_recall: 0.5361 - val_auc: 0.9091 - val_prc: 0.6294
Epoch 4/10
165/165 [==============================] - 725s 4s/step - loss: 0.8154 - accuracy: 0.9086 - precision: 0.9285 - recall: 0.8896 - a